In [ ]:
# 인퍼런스 설정
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="nlpbook/qa",
    max_seq_length=128,
    max_query_length=32,
)

In [ ]:
# 토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [ ]:
# 체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_path,
    map_location=torch.device("cpu"),
)

In [ ]:
# BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

In [ ]:
# BERT 모델 초기화
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering(pretrained_model_config)

In [ ]:
# 체크포인트 주입하기
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt["state_dict"].items()})

In [ ]:
# 평가 모드
model.eval()

In [1]:
# 인퍼런스
def inference_fn(question, context):
    if question and context:
        # 질문을 토큰화하고 인덱싱, max_query_length보다 길면 자르기
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
       )
       # 처리한 질문을 지문과 함께 토큰화하고 인덱싱, max_seq_length보다 길면 자르기
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            # 정답의 시작 위치와 관련된 로짓에서 가장 큰 값이 가리키는 토큰 위치를 알아내기
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            # 정답의 끝 위치와 관련된 로짓에서 가장 큰 값이 가리키는 토큰 위치를 알아내기
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            # 정답 시작부터 끝까지의 토큰 이어붙여 정답 만들기
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [ ]:
# 웹 서비스
from ratsnlp.nlpbook.qa import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()